(sec-ray-data-preprocessor)=
# Preprocessor

{numref}`sec-ray-data-transform` 介绍了通用接口 `map()` 和 `map_batches()`。对于结构化的表格类数据，Ray Data 在 `map()` 和 `map_batches()` 基础上，增加了一个高阶的 API：预处理器（Preprocessor）。[Preprocessor](https://docs.ray.io/en/latest/data/api/preprocessor.html) 是一系列特征处理操作，可与机器学习模型训练和推理更好地结合。其使用方式与 scikit-learn 的 [sklearn.preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) 非常相似，熟悉 scikit-learn 的用户可以快速迁移过来。对于非结构化数据，比如图片、视频等，仍然建议使用 `map()` 或者 `map_batches()`。

## 使用

Preprocessor 主要有 4 类操作：

1. [`fit()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessor.Preprocessor.fit.html)：计算 Ray Data `Dataset` 状态信息，比如计算某一列数据的方差或者均值。
2. [`transform()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessor.Preprocessor.transform.html)：执行转换操作。如果这个转换操作是有状态的，那必须先进行 `fit()`。
3. [`transform_batch()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessor.Preprocessor.transform_batch.html)：对一个批次数据进行转换操作。
4. [`fit_transform()`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessor.Preprocessor.fit_transform.html)：结合了 `fit()` 和 `transform()` 的一个操作，先对 `Dataset` 进行 `fit()`，再进行 `transform()`。

下面根据出租车数据集，来演示一下如何使用 Preprocessor。出租车数据是一个典型的结构化数据，里面有很多列，比如该旅程的距离，这些列可被用来作为机器学习算法的特征，而喂给机器学习模型前，需要进行特征处理。比如 [`MinMaxScaler`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.MinMaxScaler.html) 将特征进行归一化：

$$
x' = \frac{x - \min(x)}{\max(x) - \min(x)}
$$

In [3]:
import os
import shutil
import urllib.request
from typing import Any, Dict

import sys
sys.path.append("..")
from datasets import nyc_taxi

import ray

if ray.is_initialized:
    ray.shutdown()

ray.init()

KeyboardInterrupt: 

In [ ]:
from ray.data.preprocessors import MinMaxScaler

dataset_path = nyc_taxi()
ds = ray.data.read_parquet(dataset_path,
    columns=["trip_distance"])
ds.take(1)

NameError: name 'nyc_taxi' is not defined

经过 `MinMaxScaler` 归一化之后，原来的值变为一个归一化之后的值。

In [ ]:
preprocessor = MinMaxScaler(columns=["trip_distance"])
preprocessor.fit(ds)
minmax_ds = preprocessor.transform(ds)
minmax_ds.take(1)

2023-12-15 14:17:29,924	INFO split_read_output_blocks.py:101 -- Using autodetected parallelism=173 for stage ReadParquet to satisfy output blocks of size at least DataContext.get_current().target_min_block_size=1.0MiB.
2023-12-15 14:17:29,925	INFO split_read_output_blocks.py:106 -- To satisfy the requested parallelism of 173, each read task output is split into 173 smaller blocks.
2023-12-15 14:17:29,926	INFO streaming_executor.py:104 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]
2023-12-15 14:17:29,927	INFO streaming_executor.py:105 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-12-15 14:17:29,928	INFO streaming_executor.py:107 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_opti

[{'trip_distance': 1.8353531664835362e-05}]

In [ ]:
minmax_ds_ft = preprocessor.fit_transform(ds)
minmax_ds_ft.take(1)

/Users/luweizheng/anaconda3/envs/dispy/lib/python3.11/site-packages/ray/data/preprocessor.py:125: UserWarning: `fit` has already been called on the preprocessor (or at least one contained preprocessors if this is a chain). All previously fitted state will be overwritten!
  warnings.warn(
2023-12-15 14:51:29,990	INFO split_read_output_blocks.py:101 -- Using autodetected parallelism=173 for stage ReadParquet to satisfy output blocks of size at least DataContext.get_current().target_min_block_size=1.0MiB.
2023-12-15 14:51:29,993	INFO split_read_output_blocks.py:106 -- To satisfy the requested parallelism of 173, each read task output is split into 173 smaller blocks.
2023-12-15 14:51:29,995	INFO streaming_executor.py:104 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> AllToAllOperator[Aggregate] -> LimitOperator[limit=1]
2023-12-15 14:51:29,997	INFO streaming_executor.py:105 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, g

[{'trip_distance': 1.8353531664835362e-05}]

(raylet) [2023-12-15 15:24:05,019 E 6858 18306341] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-12-15_14-07-48_510959_95090 is over 95% full, available space: 49982570496; capacity: 1000240963584. Object creation will fail if spilling is required.


## 分类变量和数值变量

### 分类变量

机器学习模型无法接受分类变量，所以需要进行一些转换。{numref}`tab-categorical-data-preprocessor` 是几个处理分类变量的 Preprocessor。

```{table} 用于处理分类变量的 Preprocessor
:name: tab-categorical-data-preprocessor

|    Preprocessor   	| 变量类型 	 |                案例                	|
|:-----------------:	|:--------:	|:----------------------------------: |
|   [`LabelEncoder`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.LabelEncoder.html)  	| 无序分类 	 |           猫，狗，牛，羊           	  |
|   [`OrdinalEncoder`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.OrdinalEncoder.html)  	| 有序分类 	 | 高中，本科，硕士，博士             	   |
|   [`MultiHotEncoder`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.MultiHotEncoder.html) 	| 多分类   	 | ["喜剧", "动画"], ["悬疑", "动作"]     |
```

### 数值变量

使用下面的转换将数据进行转换，以适应特定的机器学习模型，{numref}`tab-numerical-data-preprocessor` 是几个处理数值变量的 Preprocessor。

```{table} 用于处理数值变量的 Preprocessor
:name: tab-numerical-data-preprocessor

| Preprocessor       	| 变量类型             	| 计算方式                                   	| 备注                                                     	|
|--------------------	|----------------------	|--------------------------------------------	|----------------------------------------------------------	|
| [`RobustScaler`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.RobustScaler.html)     	| 有离群值             	| $x' = \frac{x - \mu_{1/2}}{\mu_h - \mu_l}$ 	| $\mu_{1/2}$ 是中位数，$\mu_h$ 是最大值，$\mu_l$ 是最小值 	|
| [`MaxAbsScaler`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.MaxAbsScaler.html)     	| 数据稀疏             	| $x' = \frac{x}{\max{\vert x \vert}}$       	|                                                          	|
| [`PowerTransformer`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.PowerTransformer.html) 	| 将数据变为正太分布   	| Yeo-Johnson 或 Box-Cox                     	|                                                          	|
| [`Normalizer`](https://docs.ray.io/en/latest/data/api/doc/ray.data.preprocessors.Normalizer.html)       	| 需要对数据进行正则化 	| $x' = \frac{x}{\lVert x \rVert_p}$         	| $p$ 是正则方式，比如 `l1` 正则是绝对值求和               	|
```
